In [ ]:
from docksuitex import Protein, Ligand, PocketFinder, BatchVinaDocking, BatchAD4Docking
from docksuitex.utils import fetch_pdb, fetch_sdf, view_molecule, parse_vina_log_to_csv, parse_ad4_dlg_to_csv, clean_temp_folder
clean_temp_folder()
from pathlib import Path

In [ ]:
# Fetch and prepare protein
fetch_pdb(pdbid = "1HVR")

prot = Protein("1HVR.pdb")
prot.prepare()
prot.save_pdbqt("1HVR.pdbqt")

In [ ]:
# Find pockets using p2rank
pf = PocketFinder("1HVR.pdbqt")
pockets = pf.run()
# pf.save_report()
pockets

In [ ]:
# Fetch and prepare ligands
cid_list = ["228", "338", "339", "340"]
for cid in cid_list:
    fetch_sdf(cid=cid, save_to="ligands")

In [ ]:
for ligand in Path("ligands").glob("*.sdf"):
    lig = Ligand(str(ligand))
    lig.prepare(minimize="UFF")
    lig.save_pdbqt(Path("prepared_ligands") / (ligand.stem + ".pdbqt"))
    lig.view_molecule()

### Dock each ligand against a single receptor at all the predicted protein pockets


##### Using Vina

In [ ]:
protein = "1HVR.pdbqt"
ligands = list(Path("prepared_ligands").glob("*.pdbqt"))
centers = [item["center"] for item in pockets]  # Replace pockets with pockets[:3], for top 3 pockets

batch = BatchVinaDocking(
    receptor=protein,
    ligand_list=ligands,
    center_list=centers,
)

results_vina = batch.run_all(save_to="vina_batch_results")

for (lig, center), res in results_vina.items():
    print(f"Ligand {lig} at center {center} → results in {res}")

parse_vina_log_to_csv(results_dir="vina_batch_results", output_csv="vina_batch_results/vina_summary.csv")


##### Using AutoDock4

In [ ]:
protein = "1HVR.pdbqt"
ligands = list(Path("prepared_ligands").glob("*.pdbqt"))
centers = [item["center"] for item in pockets]  # Replace pockets with pockets[:3], for top 3 pockets

batch_ad4 = BatchAD4Docking(
    receptor=protein,
    ligand_list=ligands,
    center_list=centers,
)

results_ad4 = batch_ad4.run_all(save_to="ad4_batch_results")

for (lig, center), res in results_ad4.items():
    print(f"Ligand {lig} at center {center} → results in {res}")

parse_ad4_dlg_to_csv(results_dir="ad4_batch_results", output_csv="ad4_batch_results/ad4_summary.csv")
